# Imports

## Libs

In [ ]:
# Classic imports
import pandas as pd
import numpy as np

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Other imports
import tqdm

## Datasets

In [ ]:
# Load the datasets
train_name = "train_data_with_sin_ratios_with_m2_price_scaled" # nom du fichier csv à utiliser pour train
pred_name = "test_data"

train_df = pd.read_csv('CSV DATA/'+ train_name +'.csv')  # fichier csv à utiliser pour train
pred_df = pd.read_csv('CSV DATA/'+ pred_name +'.csv')  # fichier csv pour prédire

target_columns = "'prix'" # utiliser en mode "'prix', 'prix_m2', 'coubeh'"

#little printy print to see the data
print("Train shape: ", train_df.shape)
print("Prediction file shape: ", pred_df.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(columns=[target_columns]), train_df[target_columns], test_size=0.2, random_state=42)

,id,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,vue_note,...,cluster_tres_eleve,Year,Month,Day,cos_month,jardin,etage,soussol,m2_outside,m2_total
0,3600072,4,2.75,206.243032,493.311037,108.695652,97.547380,1.0,0,0,...,0,2015,3,30,-0.989992,1,1,1,493.311037,699.554069
1,6200017,3,1.00,124.489038,1982.162765,124.489038,0.000000,1.5,0,0,...,0,2014,11,12,0.004426,1,1,0,1982.162765,2106.651802
2,7600136,2,2.00,104.979562,106.651802,74.321813,30.657748,2.0,0,0,...,0,2014,7,18,0.753902,1,1,1,106.651802,211.631364
3,11200400,3,2.50,177.443330,416.945373,177.443330,0.000000,2.0,0,0,...,0,2014,9,23,-0.911130,1,1,0,416.945373,594.388703
4,11500890,3,2.50,290.784095,812.894835,290.784095,0.000000,2.0,0,0,...,0,2015,3,12,-0.989992,1,1,0,812.894835,1103.678930


In [33]:
train_new.drop(columns=['Year', 'Month', 'Day'], inplace=True)
test_new.drop(columns=['Year', 'Month', 'Day'], inplace=True)

clusters = ['cluster_tres_bas', 'cluster_bas', 'cluster_moyen', 'cluster_eleve', 'cluster_tres_eleve']
train_new.drop(columns=clusters, inplace=True)
test_new.drop(columns=clusters, inplace=True)

In [37]:
train_new.columns

Index(['id', 'prix', 'nb_chambres', 'nb_sdb', 'm2_interieur', 'm2_jardin',
       'm2_etage', 'm2_soussol', 'nb_etages', 'vue_mer', 'vue_note',
       'etat_note', 'design_note', 'annee_construction', 'annee_renovation',
       'm2_interieur_15voisins', 'm2_jardin_15voisins', 'zipcode', 'cos_month',
       'jardin', 'etage', 'soussol', 'm2_outside', 'm2_total',
       'prix_m2_interieur', 'prix_m2_total'],
      dtype='object')

In [38]:
test_new.columns

Index(['id', 'nb_chambres', 'nb_sdb', 'm2_interieur', 'm2_jardin', 'm2_etage',
       'm2_soussol', 'nb_etages', 'vue_mer', 'vue_note', 'etat_note',
       'design_note', 'annee_construction', 'annee_renovation',
       'm2_interieur_15voisins', 'm2_jardin_15voisins', 'zipcode', 'cos_month',
       'jardin', 'etage', 'soussol', 'm2_outside', 'm2_total'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scaling_data(df1, df2) :
    scaled_name1 = f"train-clusters_sinus_m2_scaled.csv"
    scaled_name2 = f"test-clusters_sinus_m2_scaled.csv"

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()

    # Scale the data
    to_scale1 = df1  # to not modify the original one
    to_scale2 = df2
    scaled_df1 = pd.DataFrame(scaler.fit_transform(to_scale1), columns=to_scale1.columns)
    scaled_df2 = pd.DataFrame(scaler.transform(to_scale2), columns=to_scale2.columns)

    scaled_df1.to_csv(scaled_name1, index=False)
    scaled_df2.to_csv(scaled_name2, index=False)
    return scaler, scaled_df1, scaled_df2

In [36]:
scaler, train_scaled, test_scaled = scaling_data(train_new, test_new)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- prix
- prix_m2_interieur
- prix_m2_total


# Train - regression pénalisée

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
def lasso_reg(X_train, y_train, df_test, var_cible):
    # Initialisation du modèle Lasso
    lasso = Lasso(alpha=0.1)

    # Entraînement du modèle
    lasso.fit(X_train, y_train)

    # On cherche X_test
    X_test = df_test[var_cible]

    # Prédiction sur l'ensemble de test
    y_pred = lasso.predict(X_test)

    return y_pred

# First train model - Random Forest

In [17]:
from datetime import datetime

def current_time_filename():
    # Obtenir la date et l'heure actuelles au format mois_jour_heure_min pour au cas où on veut spammer le kaggle
    current_time = datetime.now().strftime("%m_%d_%H_%M")

    # Et paf le nom de fichier avec la date et l'heure actuelles
    file_name = f"predictions_{current_time}"

    return file_name

In [18]:
rf = RandomForestRegressor(random_state=42)

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
    'max_depth': [10, 20, 30],#, None],  # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],  # Nombre minimal d'échantillons pour une division interne
    'min_samples_leaf': [1, 2, 4],    # Nombre minimal d'échantillons par feuille
    'bootstrap': [True, False]        # Utiliser bootstrap ou non, via ce qu'à dit le prof
}

In [23]:
# train with price m2
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

def train_with_total_m2_price(train_df, test_df):
    X_train = train_df.drop(columns=['id', 'prix', 'prix_m2_interieur', 'prix_m2_total'])
    y_train = train_df[['prix_m2_interieur', 'prix_m2_total']]

    print("X_train shape: ", X_train.shape)
    print("y_train shape: ", y_train.shape)

    grid_search.fit(X_train, y_train)

    print(f"Meilleurs paramètres : {grid_search.best_params_}")

    return grid_search.best_estimator_

def train_with_m2_price(train_df, test_df):
    X_train = train_df.drop(columns=['id', 'prix', 'prix_m2_interieur', 'prix_m2_total'])
    y_train = train_df['prix_m2_interieur']

    print("X_train shape: ", X_train.shape)
    print("y_train shape: ", y_train.shape)

    grid_search.fit(X_train, y_train)

    print(f"Meilleurs paramètres : {grid_search.best_params_}")

    return grid_search.best_estimator_

def train_with_price(train_df, test_df):
    X_train = train_df.drop(columns=['id', 'prix'])
    y_train = train_df['prix']

    grid_search.fit(X_train, y_train)

    print(f"Meilleurs paramètres : {grid_search.best_params_}")

    return grid_search.best_estimator_
    

## On cherche les meilleurs paramètres avec grid search

Meilleurs params :
Sans zip code, sans clustering des zip code, etude du prix
- Meilleurs paramètres : {'bootstrap': True, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Clustering des zips, etude du prix (52min13.9s)
- Meilleurs paramètres : {'bootstrap': True, 'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}

Clustering, etude du prix au m2 intérieur (24min14.4s)
- Meilleurs paramètres : {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}

Clustering, prix m2 total, sin et scale (18min58.1s)
- Meilleurs paramètres : {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
# best_rf_m2_price = train_with_m2_price(train_new, test_new)

In [ ]:
# best_rf_full_price = train_with_price(train_df, test_df)

In [ ]:
# best_rf_total_m2_price = train_with_total_m2_price(train_scaled, test_scaled)

X_train shape:  (17147, 27)
y_train shape:  (17147, 2)
Fitting 5 folds for each of 162 candidates, totalling 810 fits
Meilleurs paramètres : {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [47]:
def predict_for_m2(train_df, test_df):
    X_train = train_df.drop(columns=['id', 'prix', 'prix_m2_interieur', 'prix_m2_total'])
    y_train = train_df['prix_m2_interieur']

    model = RandomForestRegressor(bootstrap = True, max_depth = 20, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200, verbose=2, n_jobs=-1)

    model.fit(X_train, y_train)

    X_test = test_df.drop(columns=['id'])

    y_pred_m2 = model.predict(X_test) # la ca predit pour un prix au m2 !
    print(len(y_pred_m2)) # ca c'est pour du debug

    date_filename = current_time_filename()
    print(date_filename)

    filename = f"{date_filename}-cluster-prix-m2-1.csv"
    # Sauvegarder les résultats dans un fichier CSV avec ce nouveau nom
    test_df['prix_m2'] = y_pred_m2
    test_df['prix'] = test_df['prix_m2'] * test_df['m2_interieur']
    test_df[['id', 'prix']].to_csv(filename, index=False)

    print(f"Fichier sauvegardé sous : {filename}")

def predict_for_total_m2(train_df, test_df, estimator = False, is_scaled = False, m2_price_col = None, id_col = None):
    X_train = train_df.drop(columns=['id', 'prix', 'prix_m2_interieur', 'prix_m2_total'])
    y_train = train_df[['prix_m2_interieur', 'prix_m2_total']]
    
    if not estimator :
        model = RandomForestRegressor(bootstrap = True, max_depth = 20, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200, verbose=1, n_jobs=-1)
    else :
        model = estimator

    model.fit(X_train, y_train)

    X_test = test_df.drop(columns=['id'])

    y_pred_m2 = model.predict(X_test) # la ca predit pour un prix au m2 !
    print(len(y_pred_m2)) # ca c'est pour du debug

    #print(y_pred_m2[0]) # on voit que le premier de chaque duo est le prix au m2 interieur, l'autre au total

    y_pred_m2_array = seperate_total_and_inside(y_pred_m2)

    date_filename = current_time_filename()
    print(date_filename)

    # Sauvegarder les résultats dans un fichier CSV avec ce nouveau nom
    test_df['prix_m2_tot'] = y_pred_m2_array[1]
    test_df['prix'] = test_df['prix_m2_tot'] * test_df['m2_total']

    if is_scaled :
        filename = f"{date_filename}-cluster-prix-m2tot-DeScaled.csv"
        # Initialize the MinMaxScaler
        scaler = MinMaxScaler()

        prediction = test_df['prix']

        # Fit the MinMaxScaler
        scaler.fit(m2_price_col.values.reshape(-1, 1))
        # Reshape predictions to a 2D array (necessary for inverse_transform)
        predictions_scaled_reshaped = np.array(prediction).reshape(-1, 1)
        # Reverse the Min-Max scaling (inverse transform)
        predictions_original = scaler.inverse_transform(predictions_scaled_reshaped)

        test_df['prix'] = predictions_original
        test_df['id'] = id_col

        test_df[['id', 'prix']].to_csv(filename, index=False)
    
    else :
        filename = f"{date_filename}-cluster-prix-m2tot.csv"
        test_df[['id', 'prix']].to_csv(filename, index=False)

    print(f"Fichier sauvegardé sous : {filename}")

    return y_pred_m2_array[1]

def seperate_total_and_inside(array_to_separate):
    array_inside = []
    array_tot = []
    for c in array_to_separate :
        array_inside.append(c[0])
        array_tot.append(c[1])
    return array_inside, array_tot

In [ ]:
prediction = predict_for_total_m2(train_scaled, test_scaled, best_rf_total_m2_price, True, m2_price_col = train_new['prix_m2_total']*train_new['m2_total'], id_col = test_new['id']) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


# le code du cours de Logic

In [ ]:
# Train a regression model
#model = RandomForestRegressor(n_estimators=100, random_state=42, verbose=1, n_jobs=-1)
#model.fit(X_train, y_train)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=2, n_jobs=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [ ]:
# Prepare the test data (we don't have 'Sales_Qty' for the test set)
X_test = test_df.drop(columns=['index', 'Sales_Date'])

In [ ]:
# Make predictions
model = best_model
predictions = model.predict(X_test)

In [ ]:
# Output predictions (You can store them in a new DataFrame and save to CSV)
test_df['Predicted_Sales_Qty'] = predictions
test_df[['index', 'Predicted_Sales_Qty']].to_csv('predictions.csv', index=False)

In [ ]:
# Optionally, evaluate the model on the training set
train_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, train_predictions)
print(f"Mean Squared Error on training set: {mse}")